In [148]:
import pandas as pd
import numpy as np
from pathlib import Path

In [149]:
path = Path.cwd().parents[0] / "output/vote_result.xlsx"
df = pd.read_excel(path)

{'Berkarya': 8170.0,
 'Demokrat': 36892.0,
 'Garuda': 3161.0,
 'Gerindra': 103759.0,
 'Golkar': 39528.0,
 'Hanura': 8689.0,
 'Nasdem': 35755.0,
 'PAN': 28943.0,
 'PBB': 4043.0,
 'PDIP': 140803.0,
 'PKB': 19948.0,
 'PKP': 1451.0,
 'PKS': 94336.0,
 'PPP': 10680.0,
 'PSI': 45679.0,
 'Perindo': 15897.0}

In [162]:
def get_dapil_data(df: pd.DataFrame, dapil_no: int) -> (dict, pd.DataFrame):
    # get full data of dapil
    data = df.loc[df["dapil_no"] == dapil_no]

    partai_vote = (
        data.loc[:, ["partai", "partai_vote", "vote"]]
        .groupby(["partai"])
        .agg(partai_vote=("partai_vote", "mean"), vote=("vote", "sum"))
        .sum(axis=1)
        .to_dict()
    )

    # get calon vote per dapil
    calon_vote = (
        data.loc[:, ["partai", "nama", "vote"]]
        .sort_values(["partai", "vote"], ascending=False)
        .assign(rank=lambda df_: df_.groupby("partai").cumcount(ascending=True) + 1)
    )

    return partai_vote, calon_vote


def return_odd_number(index=0):
    number = index * 2 + 1
    return number


def get_element_count(input_list: list):
    output_list = []
    element_count = {}

    for item in input_list:
        if item not in element_count:
            element_count[item] = 1
        else:
            element_count[item] += 1
        output_list.append((item, element_count[item]))

    return output_list


def get_selected_partai(partai_vote: dict, num_selected: int, with_rank=False):
    partai_vote_copy = partai_vote.copy()
    selected_partai = []

    for round in list(range(1, num_selected + 1)):
        highest_voted = max(
            partai_vote_copy, key=lambda k: partai_vote_copy[k]
        )  # current highest voted partai
        for partai, vote in partai_vote_copy.items():
            times_selected = selected_partai.count(
                partai
            )  # how many times this partai been selected

            if partai == highest_voted:
                partai_vote_copy[partai] /= return_odd_number(
                    index=times_selected
                )  # divide by odd number
                selected_partai.append(partai)
    if with_rank:
        return get_element_count(selected_partai)
    return selected_partai


def get_selected_calon(
    calon_vote: pd.DataFrame, selected_partai: list, with_partai=False
):
    selected_calon = []
    for partai, rank in selected_partai:
        calon = calon_vote.loc[
            (calon_vote["partai"] == partai) & (calon_vote["rank"] == rank), "nama"
        ].values[0]
        selected_calon.append(calon)

    if with_partai:
        partai = [p for p, r in selected_partai]
        selected_calon = list(zip(partai, selected_calon))

    return selected_calon


partai_vote, calon_vote = get_dapil_data(df, 1) # specify dapil here
selected_partai = get_selected_partai(partai_vote, 12, with_rank=True) # specify num selected
selected_calon = get_selected_calon(calon_vote, selected_partai, with_partai=True)
selected_calon

[('PDIP', 'H. Prasetyo Edi Marsudi, S.H.'),
 ('PDIP', 'Pandapotan Sinaga, S.E., M.M.'),
 ('Gerindra', 'Iman Satria'),
 ('Gerindra', 'Dr. Dian Pratama, Sp.OG.'),
 ('PKS', 'H. Dany Anwar'),
 ('PKS', 'H. Ismail, S.Pd.'),
 ('PDIP', 'Wa Ode Herlina'),
 ('PSI', 'Idris Ahmad, S.K.M.'),
 ('PSI', 'Johannes Martuah, S.T.'),
 ('Golkar', 'Basri Baco, S.E., M.M.'),
 ('Golkar', 'Fitri Aprinasari Utami, S.E.'),
 ('Demokrat', 'Desie Christhyana Sari')]